In [6]:
import pandas as pd
import json
import gzip


def parse(path):
    g = gzip.open(path, "rb")
    for l in g:
        yield json.loads(l)


def getDF(path, stop=0):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
        if stop > 0 and i == stop:
            break
    return pd.DataFrame.from_dict(df, orient="index")

In [7]:
df = pd.DataFrame(columns=["overall", "verified", "reviewText", "summary", "sentiment"])

In [8]:
peek = 1_000_000
stop = 10
i = 0

for review in parse("./data/All_Amazon_Review.json.gz"):
    i += 1
    if stop > 0 and stop == i:
        break
    if i % peek == 0:
        print(f"Processed {i / peek:.2f} million reviews")

    if "overall" not in review or "verified" not in review or "reviewText" not in review or "summary" not in review:
        continue
    df.loc[i] = [review["overall"], review["verified"], review["reviewText"], review["summary"], 0]



In [9]:
df.head()

,overall,verified,reviewText,summary,sentiment
1,1.0,False,Alexa is not able to control my lights. If I a...,"VERY Buggy, doesn't work.",0
2,4.0,False,"Alexa works great for me so far, but I'm also ...",So Far So Good,0
3,1.0,False,Weak!!\n\nAlexa doesn't even recognize the nam...,Time waster,0
4,2.0,False,Can only control one of two bulbs from one of ...,Buggy,0
5,1.0,False,this worked great then randomly stopped. pleas...,stopped working,0


In [10]:
df.shape

(9, 5)

In [11]:
print("size of df in bytes: ", df.memory_usage(index=True).sum())

size of df in bytes:  369


In [12]:
df = getDF("./data/All_Amazon_Review.json.gz", 10)
print("size of df in bytes: ", df.memory_usage(index=True).sum())

size of df in bytes:  810


In [15]:
df.iloc[1]["reviewText"]

'Alexa works great for me so far, but I\'m also only controlling a single bulb at the moment. Turning on/off, changing colors and adjusting brightness are all easy and quick. That being said, I\'m expecting complications as I add more bulbs (hope for the best prepare for the worst, right?)\n\nI\'d speculate that some other users\' frustrations might stem from Alexa not recognizing their bulb or room names. After simplifying my bulb name to \'Lamp\' and listing it under a Living Room group (within the LIFx app), I\'ve been able to address it by either category pretty consistently.\n"Turn on/off living room lights."\n"Change lamp light to [color]"\n"Dim living room lights to [X]%"\n\nLike any new tech, you can expect growing pains and bugs early on. Be patient. This skill isn\'t perfect by any means, but I\'d say it\'s off to a decent start'